In [14]:
import numpy as np
import csv

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from os import listdir
        
        



# test_data = np.genfromtxt('./data_export_cobercat/data_export_1-2015.csv',skip_header=1,delimiter=',', dtype=None, names=['timestamp','date','lat','lng','signal_inst','signal_min','signal_max','signal_avg','carrier','fullCarrier','status','net','net_type','lac','cid','psc','speed,satellites','precision1','provider','activity','incident','downloadSpeed','uploadSpeed'],filling_values=(-1))

In [54]:
class cell_user:
    '''Holds data for each user'''
    
    def __init__(self, inpu):
        self.timestamp = int(inpu[0])
        self.date = inpu[1]
        self.lat = float(inpu[2])
        self.lng = float(inpu[3])
        self.signal_inst = int(inpu[4])
        self.signal_min = int(inpu[5])
        self.signal_max = int(inpu[6])
        self.signal_avg = int(inpu[7])
        self.carrier = inpu[8]
        self.fullCarrier = inpu[9]
        self.status = int(inpu[10])
            
#         self.status = int(inpu[10])
        self.net = int(inpu[11])
        self.net_type = inpu[12]
        self.lac = int(inpu[13])
        self.cid = int(inpu[14])
        self.psc = int(inpu[15])
        self.speed = float(inpu[16])
        self.satellites = int(inpu[17])
        self.precision1 = int(inpu[18])
        self.provider = inpu[19]
        self.activity = int(inpu[20])
        try:
            self.incident = inpu[21]
            if len(self.incident) == 0:
                self.incident = -1
            
        except:
            self.incident = -1
            
        try:
            self.downloadSpeed = inpu[22]
            self.uploadSpeed = inpu[23]
            
        except:
            self.downloadSpeed = -1
            self.uploadSpeed = -1
        
        
        
    def __str__(self):
        '''return the name upon str(object)'''
        return str(self.timestamp)


In [63]:
def average_attr(list_of_phones, attr):
    """averages that attribute in the list"""
    total = 0
    n = 0
    for x in list_of_phones:
        try:
            total += float(getattr(x, attr))
            n += 1
        
        except:
            pass
    
    average = total/float(n)

    return average
    

In [62]:
unix = "./data_export_cobercat/data_export_1-2015.csv"

phone_data = open(unix, "r")
phone_data_reader = csv.reader(phone_data, delimiter=',')
phone_data_list = list(phone_data_reader)
list_of_phone_data = []
error_entries = []

for x in phone_data_list[1:]:
    try:
        y = cell_user(x)
        list_of_phone_data.append (y)
    except:
        error_entries.append(str(x))
        
        
print list_of_phone_data[-1]

1448865682652


In [51]:
for x in list_of_phone_data[-10:-5]:
    print x.uploadSpeed

In [52]:
sum = 0
count = 0
for x in list_of_phone_data:
    sum += float(getattr(x, "speed"))
    count += 1

In [53]:
print sum/count

26.1284117812


In [64]:
print average_attr(list_of_phone_data, "speed")

26.1284117812
